Here I will attempt to validate new scripts that I wrote as well as collect revision history of selection of ~16 wikipedia pages

In [1]:
import sys
import pandas as pd
import os

sys.path.append('../scripts/')
import wikipedia_scripts

In [12]:
# Testing package

test_page = 'Second_moment_of_area'
path_to_data = os.path.join('../data/', '20180619_' + test_page + '/')

wikipedia_scripts.collect_consoidate_historical_edits(directory=path_to_data, wiki_title=test_page,
                                                      agent_email='williamvoje@gmail.com')

../scripts\wikipedia_scripts.py:39: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  df = pd.concat(pd.read_csv(i) for i in glob.iglob(directory + '*.csv'))


In [18]:
pd.read_csv(os.path.join(path_to_data, 'summary_' + test_page + '.csv'))

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,character_count,comment,external_link_count,heading_count,parentid,revid,timestamp,user,userid,wikilink_count,wikilinks
0,0,0,NaN,7227,Reverted edits by [[Special:Contribs/117.202.1...,2,11,846126925,846126944,2018-06-16T14:15:26Z,L293D,32436778,34,"['[[multiple integral]]', '[[physical unit|uni..."
1,1,1,NaN,7227,/* Definition */,2,11,846126757,846126925,2018-06-16T14:15:18Z,117.202.145.209,0,34,"['[[multiple integral]]', '[[physical unit|uni..."
2,2,2,NaN,7227,/* Definition */,2,11,844292537,846126757,2018-06-16T14:13:49Z,117.202.145.209,0,34,"['[[multiple integral]]', '[[physical unit|uni..."
3,3,3,NaN,7227,NaN,2,11,841071488,844292537,2018-06-04T00:06:44Z,2600:1700:F8B0:8490:7929:FC47:4FC3:CC5A,0,34,"['[[multiple integral]]', '[[physical unit|uni..."
4,4,4,NaN,7360,Reverted to revision 829153934 by [[Special:Co...,2,11,831857911,841071488,2018-05-13T22:06:06Z,93,17369702,35,"['[[multiple integral]]', '[[physical unit|uni..."
5,5,5,NaN,7343,NaN,2,11,831857744,831857911,2018-03-22T14:40:40Z,24.221.55.43,0,35,"['[[multiple integral]]', '[[physical unit|uni..."
6,6,6,NaN,7344,NaN,2,11,829153934,831857744,2018-03-22T14:39:35Z,24.221.55.43,0,35,"['[[multiple integral]]', '[[physical unit|uni..."
7,7,7,NaN,7360,NaN,2,11,821241868,829153934,2018-03-06T23:45:43Z,100.33.240.50,0,35,"['[[multiple integral]]', '[[physical unit|uni..."
8,8,8,NaN,7373,typo,2,11,816875944,821241868,2018-01-19T06:46:02Z,WereSpielChequers,4071608,35,"['[[multiple integral]]', '[[physical unit|uni..."
9,9,9,NaN,7372,NaN,2,11,816174417,816875944,2017-12-24T09:07:13Z,Victorindia87,13684355,35,"['[[multiple integral]]', '[[physical unit|uni..."


Looks like this package is working as I expected it too. Now to try significantly larger webpages.

In [21]:
list_of_pages = ['World_War_I',
                'American Civil War',
                'India',
                'Yemen' ,
                'Lady Gaga',
                'Donald Trump',
                'David Bowie',
                'Martin Luther King Jr.',
                'Google',
                '3M',
                'xHamster',
                'Reddit',
                'Vagina',
                'Human penis size',
                'Machine learning',
                'DNA_sequencing',
                'FIFA World Cup',
                'Cricket World Cup']


In [22]:
# generating_required_data

for page in list_of_pages:
    path_to_data = os.path.join('../data/', '20180619_' + page + '/')
    wikipedia_scripts.collect_consoidate_historical_edits(directory=path_to_data, wiki_title=page,
                                                      agent_email='williamvoje@gmail.com')

KeyError: 'comment'

In [24]:
# This seemed to have error when it was processing the civil war
# It also seems like I need to store hyperlinks in a differnt fashion

In [33]:
import mwapi
import mwparserfromhell
session = mwapi.Session('https://en.wikipedia.org', user_agent='williamvoje@gmail.com')

test_page = session.get(action='query', titles ='American Civil War', prop = 'revisions', 
                                rvprop='ids|flags|timestamp|comment|user|userid|content',
                                rvlimit='max')

In [34]:
wikilink_dictionary = dict()

for count, revision in enumerate(next(iter(test_page['query']['pages'].values()))['revisions']):

    # Extract edit information 
    revid, parentid, user, userid, timestamp, comment = revision_information(revision)

    # Extract information on the website itself
    parsed = mwparserfromhell.parse(revision['*'])

    character_count, external_link_count, heading_count, wikilink_count, wikilinks, wikilink_dictionary = parsed_article_metrics(parsed, wikilink_dictionary)

#     data_frame.loc[count] = [ revid, parentid, user, userid, timestamp, comment,
#                              character_count, external_link_count, heading_count, wikilink_count, wikilinks]
        

In [36]:
len(wikilinks)

737

In [30]:
def revision_information(revision_json):
    """This returns [revid, parentid, user, userid, timestamp, comment] from a
    json which details the metadata of the revision

    Args:
        revision_json (dict): json/dict structure of the revision data

    Returns:
        TYPE: list
    """
    return [revision_json['revid'], revision_json['parentid'],
            revision_json['user'], revision_json['userid'],
            revision_json['timestamp'], revision_json['comment']]


def parsed_article_metrics(parsed_article):
    """This should take a parsed article and pull out metrics of interest.
    Args:
        parsed_article (parsed): Parsed wikipedia site (mwparserfromhell)

    Returns:
        TYPE: list
    """
    character_count = len(parsed_article.strip_code())
    external_link_count = len(parsed_article.filter_external_links())
    heading_count = len(parsed_article.filter_headings())
    wikilink_count = len(set([str(link.title)
                              for link in parsed_article.filter_wikilinks()]))

    return [character_count, external_link_count, heading_count,
            wikilink_count, parsed_article.filter_wikilinks()]

In [73]:
wikilink_dictionary = dict()

wikilinks = []
wikifiles = []

rev_id = 231516516

def process_wikilinks(parsed, wikilink_dictionary):

    wikifiles = 0
    wikilinks = 0 

    for link in set([str(link.title) for link in parsed.filter_wikilinks()]):
        try:
            wikilink_dictionary[link].append(rev_id)
        except KeyError:
            wikilink_dictionary[link] = [rev_id]

        if link.startswith('File:'):
            wikifiles += 1
        else:
            wikilinks += 1
    
    return wikilinks, wikifiles, wikilink_dictionary

        
    

In [74]:
process_wikilinks(parsed, wikilink_dictionary)

(558,
 20,
 {'civil rights': [231516516],
  'New York City draft riots': [231516516],
  'History of the United States': [231516516],
  'George B. McClellan': [231516516],
  'File:Charleston Mercury Secession Broadside, 1860.jpg': [231516516],
  'File:Civil War centennial issues.jpg': [231516516],
  'Civil Rights Movement': [231516516],
  'List of memorials to Robert E. Lee': [231516516],
  'Border states (American Civil War)': [231516516],
  'Ex parte Merryman': [231516516],
  'Battle of Spotsylvania': [231516516],
  'Atlantic Ocean': [231516516],
  'Louisiana': [231516516],
  'Compromise of 1877': [231516516],
  'Simon Cameron': [231516516],
  'total war': [231516516],
  'The Good, the Bad and the Ugly': [231516516],
  'Mexican–American War': [231516516],
  'The Last Outlaw (1993 film)': [231516516],
  'Category:Wars involving the United States': [231516516],
  'William T. Sherman': [231516516],
  'Prize of war': [231516516],
  'Battle of Atlanta': [231516516],
  'Belligerent': [23151

In [75]:
import pickle

In [ ]:
pickle.dump()

In [71]:
wikilinks

558

In [67]:
string = 'file'

In [68]:
string.startswith('fi')

True

In [62]:
wikilink_dictionary['test'].append('test')

KeyError: 'test'

In [60]:
type(str(title))

str

In [61]:
set([str(link.title) for link in parsed.filter_wikilinks()])

{'12 Years a Slave (film)',
 '1804 Haiti massacre',
 '1850s in the United States',
 '1860 U.S. Presidential election',
 '1862 congressional elections',
 '54th Massachusetts Volunteer Infantry',
 '75,000 volunteers',
 'A. S. Johnston',
 'Abraham Lincoln',
 'Abraham Lincoln assassination',
 "Abraham Lincoln's first inaugural address",
 'African American',
 'African Americans in the American Civil War',
 'Agar gun',
 "Ageod's American Civil War",
 "Ageod's American Civil War II",
 'Alabama',
 'Alabama Claims',
 'Albert G. Hodges',
 'Alexandria, Louisiana',
 'Allan Gurganus',
 'Allan Nevins',
 'Alta California',
 'Ambrose Bierce',
 'Ambrose Burnside',
 'American Civil War Corps Badges',
 'American Civil War bibliography',
 'American Civil War casualties',
 'American Civil War prison camps',
 'American Civil War spies',
 'American Conqest: Divided Nation',
 'American Heritage (magazine)',
 'American Historical Review',
 'American Revolution',
 'An Occurrence at Owl Creek Bridge',
 'Anaconda

Just reworte my code. I'm going to test on the faint

In [3]:
test_page = 'Second_moment_of_area'
path_to_data = os.path.join('../data/', '20180619_' + test_page + '/')

wikipedia_scripts.collect_consoidate_historical_edits(directory=path_to_data, wiki_title=test_page,
                                                      agent_email='williamvoje@gmail.com')

../scripts\wikipedia_scripts.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  df = pd.concat(pd.read_csv(i) for i in glob.iglob(directory + '*.csv'))


In [4]:
pd.read_csv(os.path.join(path_to_data, 'summary_' + test_page + '.csv'))

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,character_count,comment,external_link_count,heading_count,parentid,revid,timestamp,user,userid,wikifile_count,wikilink_count
0,0,0,NaN,7227,Reverted edits by [[Special:Contribs/117.202.1...,2,11,846126925,846126944,2018-06-16T14:15:26Z,L293D,32436778,5,29
1,1,1,NaN,7227,/* Definition */,2,11,846126757,846126925,2018-06-16T14:15:18Z,117.202.145.209,0,5,29
2,2,2,NaN,7227,/* Definition */,2,11,844292537,846126757,2018-06-16T14:13:49Z,117.202.145.209,0,5,29
3,3,3,NaN,7227,NaN,2,11,841071488,844292537,2018-06-04T00:06:44Z,2600:1700:F8B0:8490:7929:FC47:4FC3:CC5A,0,5,29
4,4,4,NaN,7360,Reverted to revision 829153934 by [[Special:Co...,2,11,831857911,841071488,2018-05-13T22:06:06Z,93,17369702,5,30
5,5,5,NaN,7343,NaN,2,11,831857744,831857911,2018-03-22T14:40:40Z,24.221.55.43,0,5,30
6,6,6,NaN,7344,NaN,2,11,829153934,831857744,2018-03-22T14:39:35Z,24.221.55.43,0,5,30
7,7,7,NaN,7360,NaN,2,11,821241868,829153934,2018-03-06T23:45:43Z,100.33.240.50,0,5,30
8,8,8,NaN,7373,typo,2,11,816875944,821241868,2018-01-19T06:46:02Z,WereSpielChequers,4071608,5,30
9,9,9,NaN,7372,NaN,2,11,816174417,816875944,2017-12-24T09:07:13Z,Victorindia87,13684355,5,30


In [9]:
import glob
pd.concat([pd.read_csv(i) for i in glob.iglob(path_to_data + '*.csv')], 
                    sort=True)

,Unnamed: 0,character_count,comment,external_link_count,heading_count,parentid,revid,timestamp,user,userid,wikifile_count,wikilink_count
0,0,7227,Reverted edits by [[Special:Contribs/117.202.1...,2,11,846126925,846126944,2018-06-16T14:15:26Z,L293D,32436778,5,29
1,1,7227,/* Definition */,2,11,846126757,846126925,2018-06-16T14:15:18Z,117.202.145.209,0,5,29
2,2,7227,/* Definition */,2,11,844292537,846126757,2018-06-16T14:13:49Z,117.202.145.209,0,5,29
3,3,7227,NaN,2,11,841071488,844292537,2018-06-04T00:06:44Z,2600:1700:F8B0:8490:7929:FC47:4FC3:CC5A,0,5,29
4,4,7360,Reverted to revision 829153934 by [[Special:Co...,2,11,831857911,841071488,2018-05-13T22:06:06Z,93,17369702,5,30
5,5,7343,NaN,2,11,831857744,831857911,2018-03-22T14:40:40Z,24.221.55.43,0,5,30
6,6,7344,NaN,2,11,829153934,831857744,2018-03-22T14:39:35Z,24.221.55.43,0,5,30
7,7,7360,NaN,2,11,821241868,829153934,2018-03-06T23:45:43Z,100.33.240.50,0,5,30
8,8,7373,typo,2,11,816875944,821241868,2018-01-19T06:46:02Z,WereSpielChequers,4071608,5,30
9,9,7372,NaN,2,11,816174417,816875944,2017-12-24T09:07:13Z,Victorindia87,13684355,5,30


In [10]:
import pickle

In [16]:
with open('../data/20180619_Second_moment_of_area/wikilinks_to_revids.p', 'rb') as tounpick:
    test = pickle.load(tounpick)

In [17]:
test

{'Imperial units': [846126944,
  846126925,
  846126757,
  844292537,
  841071488,
  831857911,
  831857744,
  829153934,
  821241868,
  816875944,
  816174417,
  811673932],
 'beam (structure)': [846126944,
  846126925,
  846126757,
  844292537,
  841071488,
  831857911,
  831857744,
  829153934,
  821241868,
  816875944,
  816174417,
  811673932,
  810063709,
  808758625,
  807046284,
  807046184,
  806902069,
  801300665,
  801299294,
  798249184,
  798203444,
  797749711,
  797749638,
  788163453,
  773077336,
  773077275,
  771636228,
  771636222,
  768593626,
  768593549,
  765141030,
  760673684,
  758820817,
  758805564,
  755682197,
  755681407,
  755681340,
  753489097,
  753489067,
  753488761,
  747393395,
  747393342,
  745240210,
  743760332,
  743756730,
  743756677,
  743756594,
  743756541,
  743756452,
  743756307,
  743756285,
  738849283,
  729193158,
  728655105,
  722340164,
  722340119,
  716731172,
  716550769,
  716547120,
  716546847,
  716546315,
  716546119,

In [12]:
list_of_pages = ['World_War_I',
                'American Civil War',
                'India',
                'Yemen' ,
                'Lady Gaga',
                'Donald Trump',
                'David Bowie',
                'Martin Luther King Jr.',
                'Google',
                '3M',
                'xHamster',
                'Reddit',
                'Vagina',
                'Human penis size',
                'Machine learning',
                'DNA_sequencing',
                'FIFA World Cup',
                'Cricket World Cup']


In [13]:
for page in list_of_pages:
    path_to_data = os.path.join('../data/', '20180619_' + page + '/')
    wikipedia_scripts.collect_consoidate_historical_edits(directory=path_to_data, wiki_title=page,
                                                      agent_email='williamvoje@gmail.com')

The following query raised warnings: {'action': 'query', 'titles': 'World_War_I', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '', 'format': 'json'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'World_War_I', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20180426003806|838273396'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'World_War_I', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20180211162611|825118792'}
	- result -- {'*': 'Th

KeyError: '*'

In [11]:
import mwapi
import mwparserfromhell
session = mwapi.Session('https://en.wikipedia.org', user_agent='williamvoje@gmail.com')

test_page = session.get(action='query', titles ='wolrd war ' , prop = 'revisions|redirect', 
                                rvprop='ids|flags|timestamp|comment|user|userid|content',
                                rvlimit='max')

The following query raised warnings: {'action': 'query', 'titles': 'wolrd war 1', 'prop': 'revisions|redirect', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'format': 'json'}
	- query -- {'*': 'Unrecognized value for parameter "prop": redirect.'}


In [9]:
test_page

{'batchcomplete': '',
 'query': {'normalized': [{'from': 'wolrd war 1&redirects',
    'to': 'Wolrd war 1&redirects'}],
  'pages': {'-1': {'ns': 0, 'title': 'Wolrd war 1&redirects', 'missing': ''}}},
 'limits': {'revisions': 500}}

In [5]:
import glob

path_to_data = os.path.join('../data/', '20180619_' + 'World_War_I' + '/')
dataframe = pd.concat([pd.read_csv(i) for i in glob.iglob(path_to_data + '*.csv')],
                   sort=True)

In [8]:
dataframe

,Unnamed: 0,character_count,comment,external_link_count,heading_count,parentid,revid,timestamp,user,userid,wikifile_count,wikilink_count
0,0,158098,"reduce size, that takes up half the usable wid...",184,87,846610155,846618948,2018-06-19T21:39:52Z,GB fan,7997523,80,984
1,1,158098,NaN,184,87,846609792,846610155,2018-06-19T20:38:04Z,Roddy the roadkill,32292133,80,984
2,2,158098,NaN,184,87,846609546,846609792,2018-06-19T20:34:53Z,Roddy the roadkill,32292133,80,984
3,3,158098,Additional link,184,87,846609412,846609546,2018-06-19T20:32:52Z,Roddy the roadkill,32292133,80,984
4,4,158098,Check the talkpage,184,87,846399250,846609412,2018-06-19T20:31:41Z,Roddy the roadkill,32292133,80,983
5,5,158098,NaN,184,87,846330019,846399250,2018-06-18T14:37:15Z,Accesscrawl,32481963,80,984
6,6,158097,NaN,184,87,845898538,846330019,2018-06-18T01:34:27Z,Raku Hachijo,30582260,80,984
7,7,158097,NaN,184,87,845746959,845898538,2018-06-14T21:49:45Z,Iamthemostwanted2015x,33841947,80,984
8,8,158097,/* Progress of the war */ adjust images,184,87,845745217,845746959,2018-06-13T21:05:06Z,Laszlo Panaflex,361935,80,984
9,9,158105,/* July Crisis */ ce,184,87,845734764,845745217,2018-06-13T20:50:01Z,Laszlo Panaflex,361935,80,984


In [2]:
dataframe

NameError: name 'dataframe' is not defined

In [39]:
print(dataframe[dataframe.character_count < 5000]['comment'][45])

[[WP:AES|←]]Replaced content with '{{About|the major war of 1914–18||World War One (disambiguation)}} Leon rules your mum'


In [41]:
print(dataframe[dataframe.character_count < 5000]['comment'][37])

[[WP:AES|←]]Replaced content with '{{About|the majo
ds'


In [44]:
print(dataframe[dataframe.character_count < 5000]['comment'][39])

[[WP:AES|←]]Replaced content with 'why does people write such large stuff. ww1 is world war one and people died to save our British community, ask your teachers! or visit WWW.Thebes.com'


In [24]:
import mwapi
import mwparserfromhell
session = mwapi.Session('https://en.wikipedia.org', user_agent='williamvoje@gmail.com')

test_page = session.get(action='query', titles ='World War I' , prop = 'revisions', rvstartid = '385457141',
                                rvprop='ids|flags|timestamp|comment|user|userid|content')



In [36]:
test_page['query']['pages']['4764461']['revisions'][0] # ['title']

{'revid': 385457141,
 'parentid': 385401756,
 'user': 'Leon rules',
 'userid': 13090157,
 'timestamp': '2010-09-18T01:11:55Z',
 'comment': "[[WP:AES|←]]Replaced content with '{{About|the major war of 1914–18||World War One (disambiguation)}} Leon rules your mum'",
 'contentformat': 'text/x-wiki',
 'contentmodel': 'wikitext',
 '*': '{{About|the major war of 1914–18||World War One (disambiguation)}} Leon rules your mum'}

It's clear that people made some pretty extreme edits - like deleting the whole shebang in one go. 

In [45]:

test_page = session.get(action='query', titles ='World War I' , prop = 'revisions', rvstartid = '374983589',
                                rvprop='ids|flags|timestamp|comment|user|userid|content', rvlimit='max')

In [47]:
for thing in test_page['query']['pages']['4764461']['revisions']:
    thing['*']

KeyError: '*'

In [49]:
thing

{'revid': 374408778,
 'parentid': 374374794,
 'user': 'Irrypride',
 'userid': 12735446,
 'timestamp': '2010-07-20T01:20:17Z',
 'comment': 'Thought you might like this',
 'texthidden': '',
 'suppressed': ''}

Looks like people can remove metadata from commitments this information 

In [ ]:
list_of_pages = ['World_War_I',
                'American Civil War',
                'India',
                'Yemen' ,
                'Lady Gaga',
                'Donald Trump',
                'David Bowie',
                'Martin Luther King Jr.',
                'Google',
                '3M',
                'xHamster',
                'Reddit',
                'Vagina',
                'Human penis size',
                'Machine learning',
                'DNA_sequencing',
                'FIFA World Cup',
                'Cricket World Cup']


for page in list_of_pages:
    path_to_data = os.path.join('../data/', '20180619_' + page + '/')
    wikipedia_scripts.collect_consoidate_historical_edits(directory=path_to_data, wiki_title=page,
                                                      agent_email='williamvoje@gmail.com')

The following query raised warnings: {'action': 'query', 'titles': 'World_War_I', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '', 'format': 'json'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'World_War_I', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20180426003806|838273396'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'World_War_I', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20180211162611|825118792'}
	- result -- {'*': 'Th

The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20180525205832|842956950'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20180524084309|842728454'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20

	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20180213094730|825427052'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20180203212106|823855650'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': '

	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20171210131707|814712511'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20171209153008|814561584'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': '

	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20170711005119|790008995'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20170630163954|788292945'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': '

	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20170422231917|776733375'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20170422083148|776636994'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': '

	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20170316220412|770674589'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': 'ids|flags|timestamp|comment|user|userid|content', 'rvlimit': 'max', 'continue': '||', 'format': 'json', 'rvcontinue': '20170315222349|770518848'}
	- result -- {'*': 'This result was truncated because it would otherwise be larger than the limit of 12,582,912 bytes.'}
The following query raised warnings: {'action': 'query', 'titles': 'Donald Trump', 'prop': 'revisions', 'rvprop': '

In [2]:
# Things froze at reddit for some reason
list_of_pages = ['Reddit',
                'Vagina',
                'Human penis size',
                'Machine learning',
                'DNA_sequencing',
                'FIFA World Cup',
                'Cricket World Cup']


for page in list_of_pages:
    path_to_data = os.path.join('../data/', '20180619_' + page + '/')
    wikipedia_scripts.collect_consoidate_historical_edits(directory=path_to_data, wiki_title=page,
                                                      agent_email='williamvoje@gmail.com')